In [27]:
import pandas as pd 
import numpy as np
from math import log2
from Node import * 
from ImpurityMeasure import * 

In [28]:
PATH = '/Users/Sathiesh/MachineLearning/INF283/project_1/csv/tennis.csv' #TODO fiks

In [29]:
df = pd.read_csv(PATH)
y = df['play']

In [30]:
X = df.drop(['day','play'], axis=1)

In [31]:
X

,outlook,temp,humidity,wind
0,Sunny,Hot,High,Weak
1,Sunny,Hot,High,Strong
2,Overcast,Hot,High,Weak
3,Rain,Mild,High,Weak
4,Rain,Cool,Normal,Weak
5,Rain,Cool,Normal,Strong
6,Overcast,Cool,Normal,Strong
7,Sunny,Mild,High,Weak
8,Sunny,Cool,Normal,Weak
9,Rain,Mild,Normal,Weak


In [32]:
#Sender inn dataframe til meg 

In [33]:
def dataframe_to_numpy(X,y): 
    for column in X.columns: 
        X[column] = pd.factorize(X[column])[0]
    X = X.values #numpy array 
    y = pd.factorize(y)[0]
    return X,y

In [34]:
X,y = dataframe_to_numpy(X,y)

In [35]:
x = X[:]

In [36]:
x

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [2, 1, 0, 0],
       [2, 2, 1, 0],
       [2, 2, 1, 1],
       [1, 2, 1, 1],
       [0, 1, 0, 0],
       [0, 2, 1, 0],
       [2, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 1, 0, 1],
       [1, 0, 1, 0],
       [2, 1, 0, 1]])

In [37]:
X= X.T

In [38]:
#Hvor mange true og hvor mange false 
#Returnerer en dictinary
def count_values(y): 
    result = {} 
    for value in y: 
        if value not in result: result[value] = 1
        else: result[value]+=1
    return result

In [39]:
def entropy(y):
    result = count_values(y)
    e = 0
    for val in result.values(): 
        p = float(val/len(y)) 
        e = e -p*log2(p)
    return e

In [40]:
print(entropy(y))

0.9402859586706311


In [41]:
result = getattr(ImpurityMeasure(), 'entropy')(y)

In [42]:
def gini(y): 
    result = count_values(y)
    g = 1
    for val in result.values(): 
        p = float(val/len(y))
        g = g - (p)**2
    return g

In [43]:
def find_best_split(categories, y): 
    entropy_src = entropy(y)
    best_ig_score = []
    for cat in categories:
        #KOMMENTAR en cat kan typisk være outlook
        #EN CAT KAN INNEHOLDE VARIABLENE: sunny, overcast, rain 
        variables_dic = {}
        entropy_of_branches = 0
        
        #Variabel og y veridene til den 
        for index,var in enumerate(cat): 
            if var not in variables_dic: variables_dic[var] = []
            variables_dic[var].append(y[index])
 
        #Regne ut entropien for hver av variablene og legge dem sammen
        for key, var_y in variables_dic.items(): 
            result = entropy(var_y)*(len(var_y)/len(y))
            entropy_of_branches += result
        
        information_gain = entropy_src - entropy_of_branches 
        best_ig_score.append(information_gain)
    return np.argmax(best_ig_score)

In [44]:
import operator #TODO Kan jeg bruke denne? 
def find_dominant_value(values_dic): 
    return max(values_dic, key=values_dic.get)

In [56]:
def create_tree(X,y, node): 
    #Find best category 
    if len(X) == 0: 
        return node  #hva skal jeg retunere?????  
    else: 
        index = find_best_split(X,y)
        node.category = index
        node.data = X[index] #add the category data to the node 
        child_dic = count_values(node.data) #count num variables for the category 
    
        for i in child_dic.keys(): #Add variable name and Node as child to the Node
            node.add_child(i,Node()) #Verdi[0] variable navn som er gitt som tall og [1] er selve noden
    
        for var, child_node in node.children.items():
            child_node.parent_node = node
            indices = [i for i, x in enumerate(node.data) if x == var] #indeksene til variablene til i datasettet til forelder noden
            result = count_values(y[indices]) #sjekker hvor mange true false man har for variabelen
            if len(result) == 1: 
                child_node.data = next(iter(result.keys())) #Alle er like, så vi gir verdi 
                child_node.isLeaf = True
            
            elif len(X) == 1: #De er fortsatt ikke clean, men da må man finne den med flest dominerende
                child_node.data = find_dominant_value(result)
                child_node.isLeaf = True
                            
            if not child_node.isLeaf:
                node_X = np.delete(X, (index), axis=0)
                create_tree(node_X[:,indices],y[indices], child_node) #når jeg skriver return her, så fucker alt seg 

In [57]:
root = Node()
denne = create_tree(X,y, root)

In [59]:
root.children[0].children[0].data #Denne blir ikke leaf hvis jeg skriver return over*

0

In [48]:
def case_prediction(x,node): 
    while len(x) > 0: 
        cat_var = x[node.category]
        x = np.delete(x, node.category, axis=0)
        child_node = node.children[cat_var]
        if child_node.isLeaf: 
            return child_node.data
        node = child_node

In [49]:
def predict(x, node):
    result = []
    for i in x: 
        result.append(case_prediction(i,node))
    return np.array(result)

In [50]:
def accuracy(y, pred): 
    result = 0
    for idx, y_ in enumerate(pred): 
        if y_ == y[idx]: result+=1
    return (result/len(y))

In [51]:
y_ = predict(x,root)

In [52]:
accuracy(y,y_)

0.7857142857142857